In [ ]:
!pip install -q accelerate==0.30.0 peft==0.5.0 bitsandbytes==0.40.2 transformers==4.33.1 trl==0.5.0
!pip install -q sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.1/88.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.5 MB/s eta 0:00:00
ERROR: pip's dependency r

In [ ]:
import random
import pandas as pd
import torch
from datasets import Dataset, load_dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
)
from trl import RewardTrainer

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [ ]:
import os
import tqdm
from huggingface_hub import login
import yaml
with open("/content/data.yaml", 'r') as file:
    credentials = yaml.safe_load(file)

# Extract credentials
HUGGINGFACE_TOKEN = credentials["HUGGINGFACE_TOKEN"]
WANDB_API_KEY = credentials["WANDB_API_KEY"]

In [ ]:
from huggingface_hub import login
login(token=HUGGINGFACE_TOKEN)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
dataset = load_dataset('json', data_files='/content/sample_data2.json', split='train')

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['prompt', 'chosen_response', 'label', 'rejected_response'],
    num_rows: 1026
})

In [ ]:
# responses = ["response-1", "response-2"]

# # Function to randomly select chosen and not chosen responses
# def get_chosen_and_not_chosen(record, responses):
#     """
#     Given a record and list of responses, randomly selects one and returns it along with the non-selected response.

#     Args:
#         record (dict): A dictionary containing the record data.
#         responses (list): List of response keys.

#     Returns:
#         tuple: A tuple containing the chosen response, non-chosen response, and the index of the chosen response.
#     """
#     chosen_id = random.randint(0, len(responses) - 1)
#     not_chosen_id = 1 - chosen_id
#     return record[responses[chosen_id]], record[responses[not_chosen_id]], chosen_id

# rows = []

# # Iterate through the dataset
# for record in dataset:
#     chosen, not_chosen, chosen_id = get_chosen_and_not_chosen(record, responses)

#     # Append a new row to the 'rows' list
#     rows.append(
#         {
#             "instruction": record["prompt"],
#             "chosen_response": chosen,
#             "rejected_response": not_chosen,
#         }
#     )
# # Convert to a DataFrame
# df_new = pd.DataFrame(rows)

In [ ]:
# df_new

In [ ]:
# hf_dataset = Dataset.from_pandas(df_new)
# hf_dataset

In [ ]:
from transformers import (
    AutoModelForSequenceClassification,
    BitsAndBytesConfig,
    AutoTokenizer,
)
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
# Load tokenizer for the "facebook/opt-350m" model
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Prepare quantization parameters
quantization_config = BitsAndBytesConfig(load_in_8bit=False, load_in_4bit=True)

# Initialize the sequence classification model
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    quantization_config=quantization_config,  # Apply the quantization configuration
    device_map="auto",  # Assign the model to device 0
    trust_remote_code=True,  # Trust remote code
    num_labels=1,  # Set the number of labels for classification (in this case, 1)
)

# Disable cache in model configuration
model.config.use_cache = False

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at TinyLlama/TinyLlama-1.1B-Chat-v1.0 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id

# Define a formatting function for processing examples
def formatting_func(record):
    kwargs = {
        "padding": "max_length",
        "truncation": True,
        "max_length": 512,
        "return_tensors": "pt",
    }

    # Prepend the instruction and a line break to the chosen_response and rejected_response fields.
    prompt_plus_chosen_response = record["prompt"] + "\n" + record["chosen_response"]
    prompt_plus_rejected_response = record["prompt"] + "\n" + record["rejected_response"]

    # Tokenize the modified fields.
    tokens_chosen = tokenizer.encode_plus(prompt_plus_chosen_response, **kwargs)
    tokens_rejected = tokenizer.encode_plus(prompt_plus_rejected_response, **kwargs)

    return {
        "input_ids_chosen": tokens_chosen["input_ids"][0],
        "attention_mask_chosen": tokens_chosen["attention_mask"][0],
        "input_ids_rejected": tokens_rejected["input_ids"][0],
        "attention_mask_rejected": tokens_rejected["attention_mask"][0],
    }

formatted_dataset = dataset.map(formatting_func)

Map:   0%|          | 0/1026 [00:00<?, ? examples/s]

In [ ]:
formatted_dataset

Dataset({
    features: ['instruction', 'chosen_response', 'rejected_response', 'input_ids_chosen', 'attention_mask_chosen', 'input_ids_rejected', 'attention_mask_rejected'],
    num_rows: 5
})

In [ ]:
formatted_dataset = formatted_dataset.train_test_split()

In [ ]:
formatted_dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen_response', 'label', 'rejected_response', 'input_ids_chosen', 'attention_mask_chosen', 'input_ids_rejected', 'attention_mask_rejected'],
        num_rows: 769
    })
    test: Dataset({
        features: ['prompt', 'chosen_response', 'label', 'rejected_response', 'input_ids_chosen', 'attention_mask_chosen', 'input_ids_rejected', 'attention_mask_rejected'],
        num_rows: 257
    })
})

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

In [ ]:
device


device(type='cuda')

In [ ]:

from transformers import TrainingArguments
from peft import LoraConfig
from trl import RewardTrainer

# Prepare training parameters
training_args = TrainingArguments(
    output_dir="./train_logs",  # Output folder
    max_steps=100,  # Maximum number of training steps
    per_device_train_batch_size=4,  # Batch size per GPU for training
    learning_rate=1.0e-4,  # Learning rate
    optim="adamw_torch",  # Optimizer
    save_steps=50,  # How often to save checkpoints
    logging_steps=10,  # How often to log training information
    report_to="tensorboard",  # Reporting method (in this case, TensorBoard)
    remove_unused_columns=False,  # Whether to remove unused columns
    evaluation_strategy="steps",  # Evaluation strategy
    num_train_epochs=5,  # Number of training epochs
)

# Prepare PEFT parameters
peft_config = LoraConfig(
    r=16,  # Value of r
    lora_alpha=16,  # Value of lora_alpha
    bias="none",  # Bias setting
    task_type="SEQ_CLS",  # Task type (Sequence Classification)
    modules_to_save=["scores"],  # Modules to save and used to generate scores ,so it can predict score after training for new samples.
)
# Prepare RewardTrainer
trainer = RewardTrainer(
    model=model,  # The model for reinforcement learning
    tokenizer=tokenizer,  # The tokenizer for processing input data
    args=training_args,  # Training arguments
    train_dataset=formatted_dataset["train"],  # Training dataset
    eval_dataset=formatted_dataset["test"],  # Evaluation dataset
    peft_config=peft_config,  # PEFT configuration
    max_length=512,  # Maximum length of input
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None)
  warnings.warn(


In [ ]:
# Execute training
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2436: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
10,0.688500,0.622623,0.618677
20,0.477600,0.403511,0.856031
30,0.298500,0.249011,0.918288
40,0.371400,0.173910,0.941634
50,0.155600,0.134183,0.949416
60,0.117200,0.114118,0.949416
70,0.095100,0.101411,0.953307
80,0.129300,0.092321,0.957198
90,0.047300,0.088548,0.961089


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2436: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
10,0.688500,0.622623,0.618677
20,0.477600,0.403511,0.856031
30,0.298500,0.249011,0.918288
40,0.371400,0.173910,0.941634
50,0.155600,0.134183,0.949416
60,0.117200,0.114118,0.949416
70,0.095100,0.101411,0.953307
80,0.129300,0.092321,0.957198
90,0.047300,0.088548,0.961089
100,0.054800,0.087428,0.961089


TrainOutput(global_step=100, training_loss=0.24352709501981734, metrics={'train_runtime': 2265.5736, 'train_samples_per_second': 0.177, 'train_steps_per_second': 0.044, 'total_flos': 0.0, 'train_loss': 0.24352709501981734, 'epoch': 0.52})

In [ ]:
trainer.model.save_pretrained("./reward_model")

In [ ]:
import torch
def get_score(model, tokenizer, prompt, response):
    """
    Computes a score for a given prompt and response using a provided model and tokenizer.

    Args:
        model (nn.Module): The model for scoring.
        tokenizer: The tokenizer for processing input data.
        prompt (str): The prompt text.
        response (str): The response text.

    Returns:
        float: The computed score.
    """
    # Tokenize the input sequences
    inputs = tokenizer.encode_plus(
        prompt,
        response,
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors="pt",
    ).to("cuda:0")

    # Perform forward pass
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract the logits
    logits = outputs.logits

    return logits.item()

In [ ]:
x=0
dataset[x]

{'prompt': 'How did you mess up today?',
 'chosen_response': "TIFU by brushing with my mom's toothbrush.",
 'label': 'chosen',
 'rejected_response': 'Brush Teeth with Baking Soda without research.'}

In [ ]:
prompt = dataset[x]["prompt"]
rejected_response = dataset[x]["rejected_response"]
chosen_response = dataset[x]["chosen_response"]

# Get the score for the example with the less preferred response
score_less_pref = get_score(model, tokenizer, prompt, rejected_response)
print(f"Score for less preferred response: {score_less_pref}")

# Get the score for the example with the preferred response
score_pref = get_score(model, tokenizer, prompt, chosen_response)
print(f"Score for preferred response: {score_pref}")

Score for less preferred response: 2.335449695587158
Score for preferred response: 3.588369131088257


In [ ]:
prompt = dataset[2]["prompt"]
rejected_response = dataset[2]["rejected_response"]
chosen_response = dataset[2]["chosen_response"]

# Get the score for the example with the less preferred response
score_less_pref = get_score(model, tokenizer, prompt, rejected_response)
print(f"Score for less preferred response: {score_less_pref}")

# Get the score for the example with the preferred response
score_pref = get_score(model, tokenizer, prompt, chosen_response)
print(f"Score for preferred response: {score_pref}")

Score for less preferred response: 1.7976405620574951
Score for preferred response: 2.548110008239746
